In [228]:
import requests
import requests_cache
import json
import pandas as pd
import time
from IPython.core.display import clear_output
import fnmatch
requests_cache.install_cache()

In [220]:
def feed_data(sensor_id):
    """Gets all data points for a given sensor from ThingSpeak API and returns
    them as a formatted Pandas dataframe. 

    Parameters
    ----------
    sensor_id : int
        ID for sensor of interest, can be found at 
        https://forecast-dot-airqo-250220.appspot.com/api/v1/forecast/channels

    Returns
    -------
    feeds : DataFrame
        DataFrame of all responses for a given sensor from ThingSpeak API      
    """

    # Get list of responses from API
    responses = get_feeds(sensor_id)
    # Create data frame excluding last feed to avoid overlap
    # List is created backwards from responses to be chronologically correct
    frames = [pd.DataFrame(response.json()['feeds'][:-1])
              for response in responses[::-1]]
    feeds = pd.concat(frames)
    # Set row names as date of creation
    feeds = feeds.set_index('created_at')
    # Get headers from "channel" response
    feeds = get_headers(feeds, responses)

    return feeds

In [221]:
def get_feeds(sensor_id):

    new_end = ""  # Initialise variable to hold end date
    responses = []  # Initialise list of responses
    page_no = 1  # Page no. to keep count

    while True:
        # Outputs page number and then clears output
        print("Requesting page {}".format(page_no))
        clear_output(wait=True)
        page_no += 1
        # Pull 8000 requests from thingspeak API and append to responses
        r = requests.get(
            "https://api.thingspeak.com/channels/{}/feeds.json?results={}&end={}".format(sensor_id, "8000", new_end))
        responses.append(r)
        # Return error code if API request fails
        if r.status_code != 200:
            print(r.text)
            break
        # If less than 8000 observations are returned, all data points
        # are downloaded and loop ends with message to user
        if len(r.json()["feeds"]) < 8000:
            print("All items returned")
            return responses
        # Otherwise, define new_end as first date in response
        else:
            # Returns first date in response, stripped of "T" and "Z" chars
            new_end = r.json()['feeds'][0]['created_at'][:-
                                                         1].replace("T", "%20")
            # If response is got from cache, sleep to prevent API overload
            if getattr(responses, 'from_cache', False):
                time.sleep(0.25)

In [222]:
def get_headers(feeds, responses):

    # Find field column headers
    matching = fnmatch.filter(feeds.columns, "field*")
    headers = {}  # Initiate headers dict

    # Compare header names with channel keys, which describes
    # the returned field names. Populates headers with field
    # headings descriptions from channel response
    for i in matching:
        if i in responses[0].json()["channel"].keys():
            headers[str(i)] = responses[0].json()["channel"][str(i)]

    # Rename columns
    feeds = feeds.rename(headers, axis="columns")

    return(feeds)

In [223]:
feeds = feed_data(870146)

All items returned


In [226]:
feeds.head()

,entry_id,Sensor1 PM2.5_CF_1_ug/m3,Sensor1 PM10_CF_1_ug/m3,Sensor2PM2.5_CF_1_ug/m3,Sensor2 PM10_CF_1_ug/m3,Latitude,Longitude,Battery Voltage,GpsData
created_at,,,,,,,,,
2019-09-23T14:32:18Z,1,43.95,54.10,46.20,58.57,0.000000,0.000000,4.14,"0.000000,0.000000, 0.00,0.00,0.00,0.00,0.00,0.00"
2019-09-23T14:33:25Z,2,42.37,52.85,45.65,56.45,1000.000000,1000.000000,4.20,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
2019-09-23T14:34:43Z,3,42.65,52.87,45.50,56.78,1000.000000,1000.000000,4.19,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
2019-09-23T14:35:55Z,4,42.70,54.05,45.05,55.85,1000.000000,1000.000000,4.19,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
2019-09-23T14:37:22Z,5,42.28,51.45,44.42,55.60,1000.000000,1000.000000,4.16,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."


In [214]:
feeds.rename(
    {"2019-09-23T14:32:18Z": "2019-09-23T14:32:18Z".replace("T", " ")}, axis=0)

,entry_id,Sensor1 PM2.5_CF_1_ug/m3,Sensor1 PM10_CF_1_ug/m3,Sensor2PM2.5_CF_1_ug/m3,Sensor2 PM10_CF_1_ug/m3,Latitude,Longitude,Battery Voltage,GpsData
created_at,,,,,,,,,
2019-09-2314:32:18Z,1,43.95,54.10,46.20,58.57,0.000000,0.000000,4.14,"0.000000,0.000000, 0.00,0.00,0.00,0.00,0.00,0.00"
2019-09-23T14:33:25Z,2,42.37,52.85,45.65,56.45,1000.000000,1000.000000,4.20,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
2019-09-23T14:34:43Z,3,42.65,52.87,45.50,56.78,1000.000000,1000.000000,4.19,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
2019-09-23T14:35:55Z,4,42.70,54.05,45.05,55.85,1000.000000,1000.000000,4.19,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
2019-09-23T14:37:22Z,5,42.28,51.45,44.42,55.60,1000.000000,1000.000000,4.16,"1000.000000,1000.000000,1000000.00,-1.00,255.0..."
...,...,...,...,...,...,...,...,...,...
2020-02-11T11:39:23Z,110875,42.90,51.95,46.68,58.30,0.332330,32.569897,4.21,"0.332330,32.569897,1225.00,0.00,13.00,70.00,0...."
2020-02-11T11:40:45Z,110876,44.05,54.00,46.62,57.97,0.332340,32.569893,4.21,"0.332340,32.569893,1225.20,0.00,10.00,80.00,0...."
2020-02-11T11:42:14Z,110877,40.62,49.07,44.85,55.33,0.332352,32.569885,4.21,"0.332352,32.569885,1225.60,0.00,12.00,70.00,0...."


In [216]:
type(feeds)

pandas.core.frame.DataFrame